**Script Libraries & Modules**

In [ ]:
import requests
import pandas as pd
from pytz import timezone
from datetime import datetime 
from functools import reduce
import hashlib
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import BooleanType, TimestampType, StringType, StructType, StructField
from pyspark.sql.window import Window

**NBA API Call & Data Transformations**

In [ ]:
# Initializing Spark Session to use Spark for data processing and transformation
spark = SparkSession.builder.appName("NBA-Schedule-Transformation").getOrCreate() 

# Make KeyVault call to get secrets
NBA_API_Key = dbutils.secrets.get(scope="ServiceAdminKeyVault", key="example_key")  # Placeholder for NBA API Key

# NBA API Call
current_year = datetime.now().year
current_month = datetime.now().month

if current_month >= 10:  # Season starts in the current year if it's after October
    season = f"{current_year}-{str(current_year + 1)[-2:]}"
else:  # Season started in the previous year if before October
    season = f"{current_year - 1}-{str(current_year)[-2:]}"

# NBA API Call
url = 'https://example.api/nba/schedule/full'  # Placeholder for API endpoint
query = {
    'leagueId': 'example_league_id',  # Placeholder
    'season': season,                 # Using dynamically calculated season
    'teamId': 'example_team_id'       # Placeholder
}
headers = {'X-API-Key': 'example_api_key'}  # Placeholder for header key

# Fetch and handle the API response for errors
try:
    response = requests.get(url, headers=headers, params=query)
    response.raise_for_status()  # Raises an HTTPError for bad responses
    data = response.json()

    # Extract only the necessary parts from the JSON response
    games = data['leagueSchedule']['gameDates']
    
except requests.exceptions.RequestException as e:
    print(f"Error fetching NBA data: {e}")
    data = None 

# Extracting & Flattening the nested data from JSON for each game
flattened_data = [game for game_date in data.get('leagueSchedule', {}).get('gameDates', []) for game in game_date.get('games', [])]

# Defining schema for Spark DataFrame
schema = StructType([
    StructField("awayTeam", StructType([StructField("teamName", StringType(), True)]), True),
    StructField("homeTeam", StructType([StructField("teamName", StringType(), True)]), True),
    StructField("gameDateEst", StringType(), True),
    StructField("gameTimeEst", StringType(), True),
    StructField("arenaName", StringType(), True)
])

# Create Spark DataFrame directly from flattened JSON data
df_spark = spark.createDataFrame(flattened_data, schema=schema)

# Select and rename required columns, performing date and time extraction and conversion to MST
df_spark = df_spark.select(
    F.col('awayTeam.teamName').alias('awayTeam'),
    F.col('homeTeam.teamName').alias('homeTeam'),
    F.col('arenaName').alias('Venue'),
    F.split(F.col('gameDateEst'), 'T').getItem(0).alias('EventDate'),  # Extracting date part

    # Convert time from EST to MST and extract only the time component
    F.date_format(
        F.from_utc_timestamp(
            F.to_utc_timestamp(
                F.concat(
                    F.split(F.col('gameDateEst'), 'T').getItem(0),  # Date part
                    F.lit(' '), 
                    F.split(F.col('gameTimeEst'), 'T').getItem(1)   # Time part
                ),
                'EST'  # Convert from EST to UTC
            ),
            'MST'  # Convert from UTC to MST
        ), 'HH:mm:ss'  # Include time in 'HH:mm:ss' format
    ).alias('EventTime')  # Final column for converted event time
)

# Define team mappings for standard team names
team_mappings = {
    'Celtics': 'Boston Celtics', 
    'Warriors': 'Golden State Warriors', 
    'Nets': 'Brooklyn Nets', 
    'Knicks': 'New York Knicks', 
    '76ers': 'Philadelphia 76ers', 
    'Raptors': 'Toronto Raptors', 
    'Bulls': 'Chicago Bulls', 
    'Cavaliers': 'Cleveland Cavaliers', 
    'Pistons': 'Detroit Pistons', 
    'Pacers': 'Indiana Pacers', 
    'Bucks': 'Milwaukee Bucks', 
    'Nuggets': 'Denver Nuggets', 
    'Timberwolves': 'Minnesota Timberwolves', 
    'Thunder': 'Oklahoma City Thunder', 
    'Trail Blazers': 'Portland Trail Blazers', 
    'Jazz': 'Utah Jazz', 
    'Clippers': 'LA Clippers', 
    'Lakers': 'Los Angeles Lakers', 
    'Suns': 'Phoenix Suns', 
    'Kings': 'Sacramento Kings', 
    'Hawks': 'Atlanta Hawks', 
    'Hornets': 'Charlotte Hornets', 
    'Heat': 'Miami Heat', 
    'Magic': 'Orlando Magic', 
    'Wizards': 'Washington Wizards', 
    'Mavericks': 'Dallas Mavericks', 
    'Rockets': 'Houston Rockets', 
    'Grizzlies': 'Memphis Grizzlies', 
    'Pelicans': 'New Orleans Pelicans', 
    'Spurs': 'San Antonio Spurs'
}

# Convert dictionary to Spark map expression to map team short names to full names
team_map_expr = F.create_map([F.lit(k) for pair in team_mappings.items() for k in pair])

# Apply team mappings and select relevant columns for final transformation
df_spark = df_spark.select(
    'EventDate',
    'Venue',
    F.coalesce(team_map_expr[F.col('homeTeam')], F.col('homeTeam')).alias('Home'),
    F.coalesce(team_map_expr[F.col('awayTeam')], F.col('awayTeam')).alias('Away'),
    'EventTime'
)

# Create the required columns and format title as 'Home vs. Away
df_spark = df_spark.select(
    'EventDate', 
    'EventTime', 
    F.concat(F.col('Away'), F.lit(' vs. '), F.col('Home')).alias('Title'),
    'Away', 
    'Home', 
    'Venue',
    F.lit('NBA').alias('Source'),
    F.lit('onsale').alias('EventStatus')
)

# Create the final standardized DataFrame
df_nba_spark = df_spark

# display(df_nba_spark)

EventDate,EventTime,Title,Away,Home,Venue,Source,EventStatus
2024-10-04,10:00:00,Boston Celtics vs. Denver Nuggets,Boston Celtics,Denver Nuggets,Etihad Arena,NBA,onsale
2024-10-06,08:00:00,Denver Nuggets vs. Boston Celtics,Denver Nuggets,Boston Celtics,Etihad Arena,NBA,onsale
2024-10-13,18:30:00,Phoenix Suns vs. Denver Nuggets,Phoenix Suns,Denver Nuggets,Ball Arena,NBA,onsale
2024-10-15,19:00:00,Oklahoma City Thunder vs. Denver Nuggets,Oklahoma City Thunder,Denver Nuggets,Ball Arena,NBA,onsale
2024-10-17,18:00:00,Denver Nuggets vs. Minnesota Timberwolves,Denver Nuggets,Minnesota Timberwolves,Target Center,NBA,onsale
2024-10-24,20:00:00,Oklahoma City Thunder vs. Denver Nuggets,Oklahoma City Thunder,Denver Nuggets,Ball Arena,NBA,onsale
2024-10-26,15:00:00,LA Clippers vs. Denver Nuggets,LA Clippers,Denver Nuggets,Ball Arena,NBA,onsale
2024-10-28,17:30:00,Denver Nuggets vs. Toronto Raptors,Denver Nuggets,Toronto Raptors,Scotiabank Arena,NBA,onsale
2024-10-29,17:30:00,Denver Nuggets vs. Brooklyn Nets,Denver Nuggets,Brooklyn Nets,Barclays Center,NBA,onsale
2024-11-01,19:30:00,Denver Nuggets vs. Minnesota Timberwolves,Denver Nuggets,Minnesota Timberwolves,Target Center,NBA,onsale


**NHL API Call & Data Transformations**

In [ ]:
# Initializing Spark Session to use Spark for data processing and transformation
spark = SparkSession.builder.appName("NHL-Schedule-Transformation").getOrCreate()

# NHL API Call: Define endpoint for fetching Colorado Avalanche schedule data
url = 'https://example.api/nhl/club-schedule'  # Placeholder for API endpoint
try:
    response = requests.get(url)
    response.raise_for_status()  # Raises an HTTPError for bad responses
    data = response.json()

    # Extract only the necessary parts from the JSON response
    nested_data = data.get('games', [])
    
except requests.exceptions.RequestException as e:
    print(f"Error fetching NHL data: {e}")
    data = None

# Define the schema for the Spark DataFrame
teamSchema = StructType([StructField("placeName", StructType([StructField("default", StringType(), True)]), True)])
venueSchema = StructType([StructField("default", StringType(), True)])

schema = StructType([
    StructField("gameDate", StringType(), True),
    StructField("startTimeUTC", StringType(), True),
    StructField("easternUTCOffset", StringType(), True),
    StructField("awayTeam", teamSchema, True),
    StructField("homeTeam", teamSchema, True),
    StructField("venue", venueSchema, True)
])

# Create Spark DataFrame from nested JSON data using the defined schema
df_spark = spark.createDataFrame(nested_data, schema=schema)

# Select and rename required columns, performing date and time extraction and conversion to MST
df_spark = df_spark.select(
    F.col('awayTeam.placeName.default').alias('awayTeam'),
    F.col('homeTeam.placeName.default').alias('homeTeam'),
    F.col('venue.default').alias('Venue'),
    F.split(F.col('gameDate'), 'T').getItem(0).alias('EventDate'),  # Extracting date part

    # Convert start time from UTC to America/Denver timezone
    F.date_format(
        F.from_utc_timestamp(
            F.concat(
                F.split(F.col('gameDate'), 'T').getItem(0),  # Date part
                F.lit(' '),
                F.split(F.col('startTimeUTC'), 'T').getItem(1)   # Time part
            ),
            'America/Denver'  # Automatically handle MST/MDT conversion based on the date
        ), 'HH:mm:ss'  # Include time in 'HH:mm:ss' format
    ).alias('EventTime')  # Final column for converted event time
)

# Defining the team mappings dictionary
team_mappings = {
    'Boston': 'Boston Bruins', 
    'Carolina': 'Carolina Hurricanes', 
    'Buffalo': 'Buffalo Sabres', 
    'Detroit': 'Detroit Red Wings',
    'Florida': 'Florida Panthers', 
    'Montréal': 'Montreal Canadiens', 
    'Ottowa': 'Ottawa Senators', 
    'Tampa Bay': 'Tampa Bay Lightning',
    'Toronto': 'Toronto Maple Leafs', 
    'Columbus': 'Columbus Blue Jackets', 
    'New Jersey': 'New Jersey Devils', 
    'Islanders': 'New York Islanders',
    'Rangers': 'New York Rangers', 
    'Philadelphia': 'Philadelphia Flyers', 
    'Pittsburgh': 'Pittsburgh Penguins', 
    'Washington': 'Washington Capitals',
    'Chicago': 'Chicago Blackhawks', 
    'Colorado': 'Colorado Avalanche', 
    'Dallas': 'Dallas Stars',
    'Minnesota': 'Minnesota Wild', 
    'Nashville': 'Nashville Predators', 
    'St. Louis': 'St. Louis Blues', 
    'Winnipeg': 'Winnipeg Jets',
    'Anaheim': 'Anaheim Ducks', 
    'Calgary': 'Calgary Flames', 
    'Edmonton': 'Edmonton Oilers', 
    'Los Angeles': 'Los Angeles Kings',
    'San Jose': 'San Jose Sharks', 
    'Seattle': 'Seattle Kraken', 
    'Vancouver': 'Vancouver Canucks', 
    'Vegas': 'Vegas Golden Knights'
}

# Convert dictionary to Spark map expression to map team short names to full names
team_map_expr = F.create_map([F.lit(k) for pair in team_mappings.items() for k in pair])

# Apply team mappings and select relevant columns for final transformation
df_spark = df_spark.select(
    'EventDate',
    'Venue',
    F.coalesce(team_map_expr[F.col('homeTeam')], F.col('homeTeam')).alias('Home'),
    F.coalesce(team_map_expr[F.col('awayTeam')], F.col('awayTeam')).alias('Away'),
    'EventTime'  # Use the already converted EventTime
)

# Create the required columns and format title as 'Home vs. Away
df_spark = df_spark.select(
    'EventDate',
    'EventTime',
    F.concat(F.col('Away'), F.lit(' vs. '), F.col('Home')).alias('Title'),
    'Away',
    'Home',
    'Venue',
    F.lit('NHL').alias('Source'),
    F.lit('onsale').alias('EventStatus')
)

# Create the final standardized DataFrame
df_nhl_spark = df_spark

# display(df_nhl_spark)

EventDate,EventTime,Title,Away,Home,Venue,Source,EventStatus
2024-09-23,19:00:00,Dallas Stars vs. Colorado Avalanche,Dallas Stars,Colorado Avalanche,Ball Arena,NHL,onsale
2024-09-27,18:00:00,Colorado Avalanche vs. Dallas Stars,Colorado Avalanche,Dallas Stars,American Airlines Center,NHL,onsale
2024-09-29,17:00:00,Utah vs. Colorado Avalanche,Utah,Colorado Avalanche,Ball Arena,NHL,onsale
2024-10-01,19:30:00,Vegas Golden Knights vs. Colorado Avalanche,Vegas Golden Knights,Colorado Avalanche,Ball Arena,NHL,onsale
2024-10-03,20:00:00,Colorado Avalanche vs. Vegas Golden Knights,Colorado Avalanche,Vegas Golden Knights,T-Mobile Arena,NHL,onsale
2024-10-05,17:00:00,Colorado Avalanche vs. Utah,Colorado Avalanche,Utah,Maverik Center,NHL,onsale
2024-10-09,20:00:00,Colorado Avalanche vs. Vegas Golden Knights,Colorado Avalanche,Vegas Golden Knights,T-Mobile Arena,NHL,onsale
2024-10-12,19:00:00,Columbus Blue Jackets vs. Colorado Avalanche,Columbus Blue Jackets,Colorado Avalanche,Ball Arena,NHL,onsale
2024-10-14,19:00:00,New York vs. Colorado Avalanche,New York,Colorado Avalanche,Ball Arena,NHL,onsale
2024-10-16,19:30:00,Boston Bruins vs. Colorado Avalanche,Boston Bruins,Colorado Avalanche,Ball Arena,NHL,onsale


**MLS API Call & Data Transformations**

In [ ]:
# Make KeyVault call to get secrets
MLS_API_Key = 'example_api_key'  # Placeholder for the API key

# Define schema for matches data
schema = StructType([
    StructField("home_team_name", StringType(), True),
    StructField("away_team_name", StringType(), True),
    StructField("stadium_name", StringType(), True),
    StructField("planned_kickoff_time", StringType(), True)
])

# Fetch data from MLS API, handle pagination, and load it into a Spark DataFrame
def fetch_matches_data(matches_url, headers, params):
    all_data = []  # Collects data across pages
    while True:
        response = requests.get(matches_url, headers=headers, params=params)  # Make API request
        
        # If an error occurs, log an error and break out
        if response.status_code != 200:
            print(f'Error: Unable to fetch data for params: {params}')
            break
        
        # Add the match data from the current page to the overall list
        matches_data = response.json()
        all_data.extend(matches_data.get('schedule', []))

        # Check if there's more data to fetch, if so, update the page token
        if 'next_page_token' in matches_data:
            params['page_token'] = matches_data['next_page_token']
        else:
            break

    # Return a Spark DataFrame with the compiled data, structured by the schema we set above
    return spark.createDataFrame(all_data, schema=schema)

# Define URLs and headers
url = 'https://example.api/mls/competitions'  # Placeholder for the competitions API endpoint
matches_url = 'https://example.api/mls/matches/seasons/'  # Placeholder for the matches API endpoint
headers = {'x-api-key': MLS_API_Key}

# Fetch competitions data
response = requests.get(url, headers=headers)
competitions_data = response.json()

# Start with an empty DataFrame to add match data as we loop through competitions
df_spark = spark.createDataFrame([], schema=schema)

# Go through each competition in the data
for competition in competitions_data.get('competitions', []):
    competition_id = competition['competition_id']
    competition_name = competition['competition_name']
    
    # Skip MLS Test competitions
    if competition_id == 'example_competition_id':
        continue

    # Assign team IDs for regular MLS and MLS NEXT competitions
    team_id = 'example_team_id1' if "MLS NEXT" not in competition_name else 'example_team_id2'
    
    # Fetch the most recent season for each competition
    seasons_url = f'https://example.api/mls/competitions/{competition_id}/seasons'  # Placeholder for the seasons endpoint
    seasons_response = requests.get(seasons_url, headers=headers)
    
    # If we can’t get the season data, move on to the next competition
    if seasons_response.status_code != 200:
        print(f"Error: Unable to fetch seasons data for competition_id: {competition_id}")
        continue
    
    # Find the current season by sorting the seasons and getting the latest one
    seasons_data = seasons_response.json()
    sorted_seasons = sorted(seasons_data['seasons'], key=lambda x: x['season'], reverse=True)
    current_season_id = sorted_seasons[0]['season_id']
    
    # Set up the parameters to get match data for this season
    params = {
        'match_date[gte]': f'{sorted_seasons[0]["season"]}-01-01',
        'match_date[lte]': f'{sorted_seasons[0]["season"]}-12-31',
        'competition_id': competition_id,
        'team_id': team_id
    }
    
    # Fetch match data for the season and add it to the main DataFrame
    matches_df = fetch_matches_data(matches_url + current_season_id, headers, params)
    df_spark = df_spark.unionByName(matches_df, allowMissingColumns=True)

# Rename and add columns to keep a consistent format
df_spark = df_spark.withColumnRenamed('stadium_name', 'Venue') \
                   .withColumn('Source', F.lit('MLS')) \
                   .withColumn('EventStatus', F.lit('onsale'))

# Apply team name mapping for Home and Away teams
df_spark = df_spark.withColumn('Home', 
               F.when(F.col('home_team_name') == 'CF Montréal', 'CF Montreal')
                .when(F.col('home_team_name') == 'Minnesota United', 'Minnesota United FC')
                .otherwise(F.col('home_team_name'))) \
           .withColumn('Away', 
               F.when(F.col('away_team_name') == 'CF Montréal', 'CF Montreal')
                .when(F.col('away_team_name') == 'Minnesota United', 'Minnesota United FC')
                .otherwise(F.col('away_team_name')))

# Create a title column showing the match as "Home vs Away"
df_spark = df_spark.withColumn('Title', F.concat(F.col('Away'), F.lit(' vs. '), F.col('Home')))

# Convert UTC to Mountain Time directly using PySpark functions
df_spark = df_spark.withColumn(
    'planned_kickoff_time_mountain',
    F.date_format(
        F.from_utc_timestamp(F.col('planned_kickoff_time'), 'America/Denver'), 'yyyy-MM-dd\'T\'HH:mm:ss'
    )
)

# Split 'planned_kickoff_time_mountain' into 'EventDate' and 'EventTime'
df_spark = df_spark.withColumn('EventDate', F.split(F.col('planned_kickoff_time_mountain'), 'T').getItem(0)) \
                   .withColumn('EventTime', F.split(F.col('planned_kickoff_time_mountain'), 'T').getItem(1))

# Filter out rows where the venue is still marked as 'TBC'
df_spark = df_spark.filter(F.col('Venue') != 'TBC')

# Select and finalize columns for the final, cleaned DataFrame
df_mls_spark = df_spark.select('EventDate', 'EventTime', 'Title', 'Home', 'Away', 'Venue', 'Source', 'EventStatus')

# display(df_mls_spark)

Error: Unable to fetch data for params: {'match_date[gte]': '2025-01-01', 'match_date[lte]': '2025-12-31', 'competition_id': 'MLS-COM-000001', 'team_id': 'MLS-CLU-00000J'}
Error: Unable to fetch data for params: {'match_date[gte]': '2024-01-01', 'match_date[lte]': '2024-12-31', 'competition_id': 'MLS-COM-000004', 'team_id': 'MLS-CLU-000018'}
Error: Unable to fetch data for params: {'match_date[gte]': '2024-01-01', 'match_date[lte]': '2024-12-31', 'competition_id': 'MLS-COM-000005', 'team_id': 'MLS-CLU-00000J'}
Error: Unable to fetch data for params: {'match_date[gte]': '2024-01-01', 'match_date[lte]': '2024-12-31', 'competition_id': 'MLS-COM-000007', 'team_id': 'MLS-CLU-00000J'}
Error: Unable to fetch data for params: {'match_date[gte]': '2024-01-01', 'match_date[lte]': '2024-12-31', 'competition_id': 'MLS-COM-00000K', 'team_id': 'MLS-CLU-00000J'}
Error: Unable to fetch data for params: {'match_date[gte]': '2024-01-01', 'match_date[lte]': '2024-12-31', 'competition_id': 'MLS-COM-00002S

EventDate,EventTime,Title,Home,Away,Venue,Source,EventStatus
2024-11-09,20:00:00,Colorado Rapids vs. LA Galaxy,LA Galaxy,Colorado Rapids,Dignity Health Sports Park,MLS,onsale
2024-11-01,19:30:00,LA Galaxy vs. Colorado Rapids,Colorado Rapids,LA Galaxy,Dick's Sporting Goods Park,MLS,onsale
2024-10-26,21:00:00,Colorado Rapids vs. LA Galaxy,LA Galaxy,Colorado Rapids,Dignity Health Sports Park,MLS,onsale
2024-10-06,14:00:00,Timbers2 vs. Colorado Rapids 2,Colorado Rapids 2,Timbers2,University of Denver Soccer Stadium,MLS,onsale
2024-09-27,18:00:00,Colorado Rapids 2 vs. MNUFC2,MNUFC2,Colorado Rapids 2,National Sports Center Stadium,MLS,onsale
2024-09-22,16:00:00,Colorado Rapids 2 vs. Houston Dynamo 2,Houston Dynamo 2,Colorado Rapids 2,SaberCats Stadium,MLS,onsale
2024-09-15,18:00:00,Sporting KC II vs. Colorado Rapids 2,Colorado Rapids 2,Sporting KC II,Dick's Sporting Goods Park,MLS,onsale
2024-09-08,15:00:00,St Louis CITY2 vs. Colorado Rapids 2,Colorado Rapids 2,St Louis CITY2,University of Denver Soccer Stadium,MLS,onsale
2024-09-01,18:45:00,Colorado Rapids 2 vs. North Texas SC,North Texas SC,Colorado Rapids 2,Choctaw Stadium,MLS,onsale
2024-08-23,19:00:00,Colorado Rapids 2 vs. Austin FC II,Austin FC II,Colorado Rapids 2,Parmer Field,MLS,onsale


**Ticket Master Ball Arena API Call & Data Transformations** 

In [ ]:
# API Call with error handling
try:
    response = requests.get('https://example.api/tm/venue/1')  # Placeholder for API endpoint
    response.raise_for_status()  # Raises an HTTPError for bad responses
    json_data = response.json()  # Extract the JSON data
except requests.exceptions.RequestException as e:
    print(f"Error fetching venue data: {e}")
    json_data = None  # Set json_data to None if the request fails

# Proceed only if data was successfully fetched
if json_data:
    # Define schema for the JSON data structure we’re working with
    schema = StructType([
        StructField('name', StringType(), True),
        StructField('dates', StructType([
            StructField('start', StructType([
                StructField('localTime', StringType(), True),  # Start time of the event in local time
                StructField('localDate', StringType(), True)   # Start date of the event
            ]), True),
            StructField('status', StructType([
                StructField('code', StringType(), True)
            ]), True)
        ]), True),
        StructField('sales', StructType([
            StructField('public', StructType([
                StructField('startTBD', BooleanType(), True)
            ]), True)
        ]), True)
    ])

    # Convert JSON to Spark DataFrame using the defined schema
    df_spark = spark.createDataFrame(json_data, schema)

    # Perform transformations: select, rename, and handle string operations
    df_spark = df_spark.select(
        col('name').alias('Title'),
        col('dates.start.localTime').alias('localTime'),
        col('dates.start.localDate').alias('localDate'),
        col('sales.public.startTBD').cast('boolean').alias('startTBD'),
        col('dates.status.code').alias('EventStatus')
    ).withColumn(
        'EventDate', split(col('localDate'), 'T').getItem(0)  # Extract just the date part from localDate
    ).withColumn(
        'EventTime', regexp_replace(split(col('localTime'), 'T').getItem(1), '\\+00:00', '')  # Clean up time format
    ).withColumn(
        'Away', when(col('Title').contains('Mammoth'), split(col('Title'), ' vs. ').getItem(1))
                .otherwise(when(col('Title').contains(' vs. '), split(col('Title'), ' vs. ').getItem(0)).otherwise(None))
    ).withColumn(
        # Set Mammoth as the Home team and opponent as the Away team
        'Home', when(col('Title').contains('Mammoth'), 'Colorado Mammoth')
                .otherwise(when(col('Title').contains(' vs. '), split(col('Title'), ' vs. ').getItem(1)).otherwise(None))    
    ).withColumn(
        'Venue', lit('Ball Arena')  # Set the venue name
    ).withColumn(
        'Source', lit('Ticketmaster')  # Set the source of data as Ticketmaster
    ).withColumn(
        'Title', concat(col('Away'), lit(' vs. '), col('Home'))  # Set title as "Away vs. Home"
    ).filter(
        ~(col('Title').contains('Avalanche') | col('Title').contains('Nuggets'))  # Exclude specific events
    )

    # Finalize DataFrame by selecting only the necessary columns for output
    df_ball_spark = df_spark.select('EventDate', 'EventTime', 'Title', 'Away', 'Home', 'Venue', 'Source', 'EventStatus')

    # display(df_ball_spark)

EventDate,EventTime,Title,Away,Home,Venue,Source,EventStatus
2024-12-21,19:00:00,Halifax Thunderbirds vs. Colorado Mammoth,Halifax Thunderbirds,Colorado Mammoth,Ball Arena,Ticketmaster,onsale
2024-12-29,14:00:00,Georgia Swarm vs. Colorado Mammoth,Georgia Swarm,Colorado Mammoth,Ball Arena,Ticketmaster,onsale
2025-01-11,19:00:00,Albany FireWolves vs. Colorado Mammoth,Albany FireWolves,Colorado Mammoth,Ball Arena,Ticketmaster,onsale
2025-02-01,19:30:00,Toronto Rock vs. Colorado Mammoth,Toronto Rock,Colorado Mammoth,Ball Arena,Ticketmaster,onsale
2025-02-21,19:00:00,Las Vegas Desert Dogs vs. Colorado Mammoth,Las Vegas Desert Dogs,Colorado Mammoth,Ball Arena,Ticketmaster,onsale
2025-03-01,19:00:00,Calgary Roughnecks vs. Colorado Mammoth,Calgary Roughnecks,Colorado Mammoth,Ball Arena,Ticketmaster,onsale
2025-04-05,19:00:00,Buffalo Bandits vs. Colorado Mammoth,Buffalo Bandits,Colorado Mammoth,Ball Arena,Ticketmaster,onsale
2025-04-13,14:00:00,Saskatchewan Rush vs. Colorado Mammoth,Saskatchewan Rush,Colorado Mammoth,Ball Arena,Ticketmaster,onsale


**Ticket Master Dicks Sporting Goods Park API Call & Data Transformations**

In [ ]:
# Make API call with error handling
url = 'https://example.api/tm/venue/2'  # Placeholder for API endpoint

try:
    response = requests.get(url)
    response.raise_for_status()  # Raises an HTTPError for bad responses
    json_data = response.json()  # Parse the JSON data
except requests.exceptions.RequestException as e:
    print(f"Error fetching AltTix data: {e}")
    json_data = None  # Set json_data to None to keep things from breaking if the request fails

# Check if json_data was successfully retrieved before proceeding
if json_data:
    # Define the schema that outlines the structure of the JSON data
    schema = StructType([
        StructField('name', StringType(), True),  # Event name
        StructField('dates', StructType([
            StructField('start', StructType([
                StructField('localTime', StringType(), True),  # Local start time
                StructField('localDate', StringType(), True)  # Local start date
            ]), True),
            StructField('status', StructType([
                StructField('code', StringType(), True)  # Event status code
            ]), True)
        ]), True),
        StructField('sales', StructType([
            StructField('public', StructType([
                StructField('startTBD', BooleanType(), True)  # Flag for TBD start time
            ]), True)
        ]), True)
    ])

    # Convert the JSON data to a Spark DataFrame based on our defined schema
    df_spark = spark.createDataFrame(json_data, schema)

    # Transform the DataFrame: select and rename columns, and add some custom handling for strings
    df_spark = df_spark.select(
        col('name').alias('Title'),                         # Rename event name to 'Title'
        col('dates.start.localTime').alias('localTime'),    # Get local start time
        col('dates.start.localDate').alias('localDate'),    # Get local start date
        col('sales.public.startTBD').cast('boolean').alias('startTBD'),
        col('dates.status.code').alias('EventStatus')       # Event status code
    ).withColumn(
        'EventDate', split(col('localDate'), 'T').getItem(0)    # Separate just the date portion
    ).withColumn(
        'EventTime', regexp_replace(split(col('localTime'), 'T').getItem(1), '\\+00:00', '')    # Clean up the time format
    ).withColumn(
        'Away', when(col('Title').contains(' vs. '), split(col('Title'), ' vs. ').getItem(0)).otherwise(None)
    ).withColumn(
        'Home', when(col('Title').contains(' vs. '), split(col('Title'), ' vs. ').getItem(1)).otherwise(None)
    ).withColumn(
        'Venue', lit("Dick's Sporting Goods Park")  # Set venue name for all rows
    ).withColumn(
        'Source', lit('Ticketmaster')               # Set the source as 'Ticketmaster'
    ).filter(
        ~col('Title').like('%Leagues Cup%')         # Filter out events with 'Leagues Cup' in the title
    )

    # Finalize DataFrame by selecting only the necessary columns for output
    df_dsgp_spark = df_spark.select('EventDate', 'EventTime', 'Title', 'Home', 'Away', 'Venue', 'Source', 'EventStatus')

    # Uncomment to display the final DataFrame
    # display(df_dsgp_spark)

**Ticket Master Paramount Park API Call & Data Transformations**

In [ ]:
# API Call with error handling
url = 'https://example.api/tm/venue/3'  # Placeholder for API endpoint

try:
    response = requests.get(url)
    response.raise_for_status()  # Raises an HTTPError for bad responses
    json_data = response.json()
except requests.exceptions.RequestException as e:
    print(f"Error fetching Ticketmaster Paramount Park data: {e}")
    json_data = None

# Check if json_data was successfully retrieved before proceeding
if json_data:
    # Define the schema for the relevant fields
    schema = StructType([
        StructField('name', StringType(), True),
        StructField('dates', StructType([
            StructField('start', StructType([
                StructField('localTime', StringType(), True),
                StructField('localDate', StringType(), True)
            ]), True),
            StructField('status', StructType([
                StructField('code', StringType(), True)
            ]), True)
        ]), True),
        StructField('sales', StructType([
            StructField('public', StructType([
                StructField('startTBD', BooleanType(), True)
            ]), True)
        ]), True)
    ])

    # Create Spark DataFrame directly from JSON (no pandas needed)
    df_spark = spark.createDataFrame(json_data, schema=schema)

    # Perform transformations: select, rename, and handle string operations
    df_spark = df_spark.select(
        col('name').alias('Title'),
        col('dates.start.localTime').alias('localTime'),
        col('dates.start.localDate').alias('localDate'),
        col('sales.public.startTBD').cast('boolean').alias('startTBD'),
        col('dates.status.code').alias('EventStatus')
    ).withColumn(
        'EventDate', split(col('localDate'), 'T').getItem(0)  # Extracting only the date part
    ).withColumn(
        'EventTime', regexp_replace(split(col('localTime'), 'T').getItem(1), '\\+00:00', '')  # Removing the timezone offset
    ).withColumn(
        'Away', when(col('Title').contains(' vs. '), split(col('Title'), ' vs. ').getItem(0)).otherwise(None)
    ).withColumn(
        'Home', when(col('Title').contains(' vs. '), split(col('Title'), ' vs. ').getItem(1)).otherwise(None)
    ).withColumn(
        'Venue', lit('Paramount')  # Adding constant Venue value
    ).withColumn(
        'Source', lit('Ticketmaster')  # Adding constant Source value
    )

    # Select and arrange final columns for the standardized DataFrame
    df_paramount_spark = df_spark.select('EventDate', 'EventTime', 'Title', 'Home', 'Away', 'Venue', 'Source', 'EventStatus')

    # Uncomment the following line to display the final DataFrame output
    # display(df_paramount_spark)

**Union together Master Calendar data sources, dedupe, and create staging table**

In [ ]:
# Step 1: Union all DataFrames into a master DataFrame
dataframes = [df_nba_spark, df_nhl_spark, df_mls_spark, df_ball_spark, df_dsgp_spark, df_paramount_spark]
df_master_calendar = dataframes[0]

for df in dataframes[1:]:
    df_master_calendar = df_master_calendar.unionByName(df, allowMissingColumns=True)

# Step 2: Deduplicate rows based on key columns
df_master_calendar_dedupe = df_master_calendar.dropDuplicates(['EventDate', 'EventTime', 'Title', 'Venue'])

# Step 3: Rank events with multiple times within the same day and title
window_spec = Window.partitionBy('EventDate', 'Title', 'Home', 'Away', 'Venue', 'Source').orderBy('EventTime')
df_master_calendar_ranked = df_master_calendar_dedupe.withColumn('Rank', rank().over(window_spec))

# Step 4: Pivot the DataFrame to create columns EventTime1, EventTime2, EventTime3
df_master_calendar_pivoted = df_master_calendar_ranked \
    .groupBy('EventDate', 'Title', 'Home', 'Away', 'Venue', 'Source', 'EventStatus') \
    .pivot('Rank') \
    .agg(first('EventTime'))

# Rename pivoted columns
df_master_calendar_pivoted = df_master_calendar_pivoted \
    .withColumnRenamed('1', 'EventTime1') \
    .withColumnRenamed('2', 'EventTime2') \
    .withColumnRenamed('3', 'EventTime3')

# Step 5: Ensure EventTime2 and EventTime3 exist, setting them to null if missing
for col_name in ['EventTime2', 'EventTime3']:
    if col_name not in df_master_calendar_pivoted.columns:
        df_master_calendar_pivoted = df_master_calendar_pivoted.withColumn(col_name, lit(None).cast(StringType()))

# Step 6: Format EventTime1, EventTime2, EventTime3
df_master_calendar_final = df_master_calendar_pivoted \
    .withColumn('EventTime1', col('EventTime1')) \
    .withColumn('EventTime2', col('EventTime2')) \
    .withColumn('EventTime3', col('EventTime3'))

# Step 7: Add HashKey and InsertDate
columns_for_hash = ['EventDate', 'EventTime1', 'EventTime2', 'EventTime3', 'Title', 'Venue', 'EventStatus']
df_master_calendar_final = df_master_calendar_final.withColumn(
    'HashKey', sha2(concat_ws('|', *columns_for_hash), 256)
).withColumn(
    'InsertDate', date_format(current_timestamp(), 'yyyy-MM-dd HH:mm:ss').cast(StringType())
)

# Step 8: Sort by Source with TicketMaster events first
df_master_calendar_final = df_master_calendar_final.orderBy(col('Source').desc())

# Step 9: Set catalog and schema (replace with placeholders for security)
spark.sql("USE CATALOG example_catalog")
spark.sql("USE example_schema")

# Step 10: Create a temporary staging table
df_master_calendar_final.createOrReplaceTempView("staging")

# Display final DataFrame
display(df_master_calendar_final)

EventDate,Title,Home,Away,Venue,Source,EventStatus,EventTime1,EventTime2,EventTime3,HashKey,InsertDate
0001-01-01,Nanpa Basico - USA Tour,null,null,Paramount,Ticketmaster,postponed,00:00:00,null,null,c08005d440476be7487698cf707d1fd497ede0835cf06fef7d154b9a068bbf71,2024-12-12 19:22:48
2024-12-13,The Jinkx & DeLa Holiday Show,null,null,Paramount,Ticketmaster,onsale,20:00:00,null,null,5a3f888551a513adc458b6b201ac93943de2ad083badb06717370f9c5d4e2ef7,2024-12-12 19:22:48
2024-12-14,Dane Cook: Fresh New Flavor,null,null,Paramount,Ticketmaster,onsale,19:00:00,null,null,5526ad1eef4ef2a116014e3033bcfca9bc96091e01170e414d953867933cee0b,2024-12-12 19:22:48
2024-12-15,Gregorian: Pure Chants,null,null,Paramount,Ticketmaster,cancelled,20:00:00,null,null,f4c55b6731a2249abe4605fd14e6aa0adcd9ec835ac16900dd478c0958a8ca7a,2024-12-12 19:22:48
2024-12-16,Derek Hough Dance For The Holidays,null,null,Paramount,Ticketmaster,onsale,20:00:00,null,null,28347433016fb187041c9da6b1fb84f7f996e2c1c789930a27efcc7d47364a2a,2024-12-12 19:22:48
2024-12-18,The Allman Betts Family Revival with Devon Allman & Duane Betts,null,null,Paramount,Ticketmaster,onsale,20:00:00,null,null,bcb494521313bea08afcbf0f82924500d75a0f1ea0f4d21148b69f482cab2f2f,2024-12-12 19:22:48
2024-12-21,Face Vocal Band: Joy to the World,null,null,Paramount,Ticketmaster,onsale,19:30:00,null,null,3cf99713d60c2ae78590da408a764ba4a7e0f278fb980bd0a3e58b8842a3991d,2024-12-12 19:22:48
2024-12-21,Halifax Thunderbirds vs. Colorado Mammoth,Colorado Mammoth,Halifax Thunderbirds,Ball Arena,Ticketmaster,onsale,19:00:00,null,null,9762a54e155353c0f87a03dbd180fe42171077c594e3d90ea1fccb7ca480529b,2024-12-12 19:22:48
2024-12-28,Aries Spears,null,null,Paramount,Ticketmaster,cancelled,20:00:00,null,null,867d684bdc1bc212947b798c0fdf65143714b8e5b350213adedaa80937ec5742,2024-12-12 19:22:48
2024-12-29,Georgia Swarm vs. Colorado Mammoth,Colorado Mammoth,Georgia Swarm,Ball Arena,Ticketmaster,onsale,14:00:00,null,null,1d08edb1ec3726cfbfcc6ea631cc4d9d49861a0fba30042f99d79f3f6d5a50b2,2024-12-12 19:22:48


Creating Archive Table if Necessary

In [ ]:
%sql
USE CATALOG example_catalog;
USE example_schema;
DELETE FROM calendar_archive;
-- Create the calendar_archive table if it doesn't already exist
CREATE TABLE IF NOT EXISTS calendar_archive (
    EventDate     DATE,
    Title         VARCHAR(255),
    Home          VARCHAR(255),
    Away          VARCHAR(255),
    Venue         VARCHAR(255),
    Source        VARCHAR(255),
    EventStatus   VARCHAR(255),
    EventTime1    VARCHAR(255),
    EventTime2    VARCHAR(255),
    EventTime3    VARCHAR(255),
    HashKey       VARCHAR(255),
    InsertDate    TIMESTAMP
);

**Upsert staging table into archive table**

In [ ]:
%sql
-- Merges data from the staging table into the calendar_archive table by matching records
MERGE INTO calendar_archive AS tar
USING staging AS base
ON tar.EventDate = base.EventDate
   AND tar.Title = base.Title
   AND tar.Venue = base.Venue

-- Update existing records where there is a change in the HashKey
WHEN MATCHED AND tar.HashKey <> base.HashKey THEN
UPDATE SET
   tar.EventStatus = base.EventStatus,
   tar.EventTime1 = base.EventTime1,
   tar.EventTime2 = base.EventTime2,
   tar.EventTime3 = base.EventTime3,
   tar.HashKey    = base.HashKey,
   tar.InsertDate = base.InsertDate

-- Insert new records that do not exist in the calendar_archive table
WHEN NOT MATCHED THEN
INSERT (EventDate, Title, Home, Away, Venue, Source, EventStatus, EventTime1, EventTime2, EventTime3, HashKey, InsertDate)
VALUES (base.EventDate, base.Title, base.Home, base.Away, base.Venue, base.Source, base.EventStatus, base.EventTime1, base.EventTime2, base.EventTime3, base.HashKey, base.InsertDate);

-- Delete postponed and placeholder events
DELETE FROM calendar_archive
WHERE EventStatus = 'postponed' OR EventDate = '0001-01-01';

num_affected_rows
1


**Creating Master Calendar table for Dataverse virtual table**


In [ ]:
%sql
USE CATALOG example_catalog;
USE default;

-- Create calendar_dataverse table if it doesn't already exist
create table if not exists calendar_dataverse (
    EventDate         DATE,
    EventTime         VARCHAR(10),
    EventDoors        VARCHAR(10),
    EventDateTime     VARCHAR(255),
    EventDateTimeISO  TIMESTAMP,
    Title             VARCHAR(255),
    EventTimeTitle    VARCHAR(255),
    Home              VARCHAR(255),
    Away              VARCHAR(255),
    Venue             VARCHAR(255),
    Source            VARCHAR(255),
    EventColorCode    VARCHAR(20),
    CatergoryOfEvent     STRING,
    EventID           VARCHAR(36) not null
);

-- Remove any existing data in dataverse table to allow fresh inserts
delete from calendar_dataverse;

-- Insert event records into the table for each EventTime column (EventTime1, EventTime2, EventTime3)
insert into calendar_dataverse (EventDate, EventTime, EventDoors, EventDateTime, EventDateTimeISO, Title, EventTimeTitle, Home, Away, Venue, Source, EventColorCode, CategoryOfEvent, EventID)
select 
    cast(EventDate as date) as EventDate, 
    date_format(timestampadd(HOUR, +7,from_utc_timestamp(EventTime1, 'America/Denver')), 'hh:mm a') as EventTime,
    date_format(timestampadd(HOUR, +6, from_utc_timestamp(EventTime1, 'America/Denver')), 'hh:mm a') as EventDoors,
    CONCAT(date_format(cast(EventDate as date), 'MM/dd/yyyy'), ' ', date_format(timestampadd(HOUR, +7,from_utc_timestamp(EventTime1, 'America/Denver')), 'HH:mm:ss')) as EventDateTime,
    cast(CONCAT(EventDate, ' ', date_format(timestampadd(HOUR, +7,from_utc_timestamp(EventTime1, 'America/Denver')), 'HH:mm:ss')) as TIMESTAMP) as EventDateTimeISO,
    
    -- Modify title if the event is canceled, otherwise use it as is
    case 
        when EventStatus = 'cancelled' then CONCAT('CANCELLED: ', TRIM(Title))
        else TRIM(Title) 
    end as Title,
    
    -- Create EventTimeTitle by combining time and title columns, with a 'CANCELLED' label if applicable
    CONCAT(date_format(timestampadd(HOUR,+7,from_utc_timestamp(EventTime1, 'America/Denver')), 'hh:mm a'), ' ', 
        case when EventStatus = 'cancelled' then CONCAT('CANCELLED: ', TRIM(Title)) else TRIM(Title) end) as EventTimeTitle,

    Home, 
    Away, 
    Venue, 
    Source,

    -- Apply color codes based on event venue or teams involved
        case
        when Home is null and Venue = 'Ball Arena' then 'Ball Arena'
        when Home is null and Venue = 'Paramount' then 'Paramount'
        when Home is null and Venue = 'Dicks Sporting Goods Park' then 'DSGP'
        when Title like '%Nuggets%' and Home like '%Nuggets%' then 'Nuggets Home'
        when Title like '%Nuggets%' and Away like '%Nuggets%' then 'Nuggets Away'
        when Title like '%Avalanche%' and Home like '%Avalanche%' then 'Avalanche Home'
        when Title like '%Avalanche%' and Away like '%Avalanche%' then 'Avalanche Away'
        when Title like '%Mammoth%' and Home like '%Mammoth%' then 'Mammoth Home'
        when Title like '%Mammoth%' and Away like '%Mammoth%' then 'Mammoth Away'
        when Title like '%Rapids%' and Home = 'Colorado Rapids' then 'Rapids Home'
        when Title like '%Rapids%' and Away = 'Colorado Rapids' then 'Rapids Away'
        when Title like '%Rapids 2%' and Home = 'Colorado Rapids 2' then 'Rapids 2 Home'
        when Title like '%Rapids 2%' and Away = 'Colorado Rapids 2' then 'Rapids 2 Away'
        else 'Other' 
    end as EventColorCode,

        case
        when Title like '%Nuggets%' then 'Nuggets'
        when Title like '%Avalanche%' then 'Avalanche'
        when Title like '%Mammoth%' then 'Mammoth'
        when Title like '%Rapids%' and Home = 'Colorado Rapids' then 'Rapids'
        when Title like '%Rapids%' and Away = 'Colorado Rapids' then 'Rapids'
        when Title like '%Rapids 2%' and Home = 'Colorado Rapids 2' then 'Rapids 2'
        when Title like '%Rapids 2%' and Away = 'Colorado Rapids 2' then 'Rapids 2'
        else 'Non-Sports'
        end as CategoryOfEvent,

    UUID() as EventID   -- Generate a unique EventID for each record

from calendar_archive

-- Repeat similar insertion logic for EventTime2 and EventTime3 if they contain data
union all
select 
    cast(EventDate as date) as EventDate,
    date_format(timestampadd(HOUR, +7,from_utc_timestamp(EventTime2, 'America/Denver')), 'hh:mm a') as EventTime,
    date_format(timestampadd(HOUR, +6, from_utc_timestamp(EventTime2, 'America/Denver')), 'hh:mm a') as EventDoors,
    CONCAT(date_format(cast(EventDate as date), 'MM/dd/yyyy'), ' ', date_format(timestampadd(HOUR, +7,from_utc_timestamp(EventTime2, 'America/Denver')), 'HH:mm:ss')) as EventDateTime,
    cast(CONCAT(EventDate, ' ', date_format(timestampadd(HOUR, +7,from_utc_timestamp(EventTime2, 'America/Denver')), 'HH:mm:ss')) as TIMESTAMP) as EventDateTimeISO,

    case 
        when EventStatus = 'cancelled' then CONCAT('CANCELLED: ', TRIM(Title))
        else TRIM(Title)
    end as Title,

    CONCAT(date_format(timestampadd(HOUR, +7,from_utc_timestamp(EventTime2, 'America/Denver')), 'hh:mm a'), ' ', 
        case when EventStatus = 'cancelled' then CONCAT('CANCELLED: ', TRIM(Title)) else TRIM(Title) end) as EventTimeTitle,

    Home, 
    Away, 
    Venue, 
    Source,

    -- Apply color codes based on venue for EventTime2
    case 
        when Home is null and Venue = 'Ball Arena' then 'Ball Arena'
        when Home is null and Venue = 'Paramount' then 'Paramount'
        else 'Other' 
    end as EventColorCode,
  case
        when Title like '%Nuggets%' then 'Nuggets'
        when Title like '%Avalanche%' then 'Avalanche'
        when Title like '%Mammoth%' then 'Mammoth'
        when Title like '%Rapids%' and Home = 'Colorado Rapids' then 'Rapids'
        when Title like '%Rapids%' and Away = 'Colorado Rapids' then 'Rapids'
        when Title like '%Rapids 2%' and Home = 'Colorado Rapids 2' then 'Rapids 2'
        when Title like '%Rapids 2%' and Away = 'Colorado Rapids 2' then 'Rapids 2'
        else 'Non-Sports'
    end as CategoryOfEvent,

      UUID() as EventID 

from calendar_archive 
where EventTime2 is not null

-- EventTime3
union all
select 
    cast(EventDate as date) as EventDate,
    date_format(timestampadd(HOUR, +7,from_utc_timestamp(EventTime3, 'America/Denver')), 'hh:mm a') as EventTime,
    date_format(timestampadd(HOUR, +6, from_utc_timestamp(EventTime3, 'America/Denver')), 'hh:mm a') as EventDoors,
    CONCAT(date_format(cast(EventDate as date), 'MM/dd/yyyy'), ' ', date_format(timestampadd(HOUR, +7,from_utc_timestamp(EventTime3, 'America/Denver')), 'HH:mm:ss')) as EventDateTime,
    cast(CONCAT(EventDate, ' ', date_format(timestampadd(HOUR, +7,from_utc_timestamp(EventTime3, 'America/Denver')), 'HH:mm:ss')) as TIMESTAMP) as EventDateTimeISO,

    case 
        when EventStatus = 'cancelled' then CONCAT('CANCELLED: ', TRIM(Title))
        else TRIM(Title)
    end as Title,

    CONCAT(date_format(timestampadd(HOUR, +7,from_utc_timestamp(EventTime3, 'America/Denver')), 'hh:mm a'), ' ', 
        case when EventStatus = 'cancelled' then CONCAT('CANCELLED: ', TRIM(Title)) else TRIM(Title) end) as EventTimeTitle,

    Home, 
    Away, 
    Venue, 
    Source,

    -- Apply color code logic for EventTime3
    case 
        when Home is null and Venue = 'Ball Arena' then 'Ball Arena'
        else 'Other' 
    end as EventColorCode,
    case 
        when Title like '%Nuggets%' then 'Nuggets'
        when Title like '%Avalanche%' then 'Avalanche'
        when Title like '%Mammoth%' then 'Mammoth'
        when Title like '%Rapids%' and Home = 'Colorado Rapids' then 'Rapids'
        when Title like '%Rapids%' and Away = 'Colorado Rapids' then 'Rapids'
        when Title like '%Rapids 2%' and Home = 'Colorado Rapids 2' then 'Rapids 2'
        when Title like '%Rapids 2%' and Away = 'Colorado Rapids 2' then 'Rapids 2'
        else 'Non-Sports'
    end as CategoryOfEvent,
  
    UUID() as EventID

from calendar_archive 
where EventTime3 is not null;
